In [ ]:
# This notebook is only for server (no colab func.)

import os
os.chdir("/app")

In [ ]:
# Datasets
!find /app/data/ -name "*_dataset_*"

In [ ]:
# Tests
!ls -l /app/logs/

In [ ]:
# constatnts
MAX_SEQ_LEN = 128
MIN_SEQ_LEN = int(MAX_SEQ_LEN / 2)

SOURCE = "coursera"
EMBEDDING = "bert"
N_INTENTS = 100
BALANCED = True
CUSTOM_PARAMETERS = dict()
CUSTOM_PARAMETERS_AS_STRING = "_".join([str(key) + "=" + str(value) for key, value in CUSTOM_PARAMETERS])

# Please fill these
DATASET_PATH = "/app/data/"+SOURCE+"_dataset_top"+str(N_INTENTS)+"/"
TEST_NAME = "dataset={}_embedding={}{}{}intents={}".format(
    SOURCE, 
    EMBEDDING, 
    "_balanced_" if BALANCED else "_", 
    CUSTOM_PARAMETERS_AS_STRING + "_" if CUSTOM_PARAMETERS_AS_STRING != "", 
    N_INTENTS
)

PRETRAINED_MODEL_PATH = "./models/bert/"
# modify if model changed to other H-XXXX!!!!
OUTPUT_SIZE = 1024

In [ ]:
np.random.seed(1234)
tf.set_random_seed(1234)

In [ ]:
test_dataset = pd.read_csv(os.path.join(DATASET_PATH, "test.csv"))

In [ ]:
# Preprocess texts
bp = BertPreprocessor(max_sequence_length=MAX_SEQ_LEN, pretrained_model_path=PRETRAINED_MODEL_PATH)
X_test, y_test = bp.fit_transform_classification(dataset["Text"].values, dataset["Label"].values)

In [ ]:
# create model from saved configs and load best weights
with tf.keras.utils.CustomObjectScope({'Bert': Bert}):
    with open(os.path.join(os.path.join("/app/logs/", TEST_NAME), "configs.json")) as file:
        json_str = json.load(file)
        model = tf.keras.models.model_from_json(json_str)
model.load_weights(os.path.join(os.path.join("/app/logs/", TEST_NAME), "weights.hdf5"))

In [ ]:
# Generate predictions
preds = model.predict(x=X_test)

In [ ]:
# save predictions
np.save(
    os.path.join(os.path.join("/app/logs/", TEST_NAME), "predictions.npy"), 
    preds
)